<a href="https://colab.research.google.com/github/aishwaryasharmaccoew/CIS6930-NLP/blob/aishwarya/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSA**

**Loading Data**

In [17]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
#url = 'https://raw.githubusercontent.com/aishwaryasharmaccoew/CIS6930-NLP/aishwarya/Data/addiction_2018_features_tfidf_256_clean.csv'
url= "/content/sample_data/depression_post_features_tfidf_256.csv"
df1 = pd.read_csv(url)
df= df1[['post']].copy()
df.head()

,post
0,My heart aches but i cant break Recently my re...
1,Cheers I’m not sure if this is the right sub t...
2,Fuck titles. I’m depressed I feel so fucking w...
3,suffering from low energy I'm not exaggerating...
4,I can already tell This year is just going to ...


**Data Preprocessing** <br>
Steps involved are :
1. Remove all the special characters from the text.
2. Remove all the words with less than 5 letters.
3. Lowercase all the characters.
4. Remove stop words.
5. Remove profanity


In [18]:
#remove special characters
df['preprocessed_posts'] = df['post'].str.replace("[^a-zA-Z#]", " ")
#remove words have letters less than 5
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>5]))
#lowercase all characters
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: x.lower())
#remove stop words

# Remove profanity

import re
pattern = r'\b' + "fuck" + r'\w*\b'
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: re.sub(pattern, '', x))


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# tokenization
tokenized_doc = df['preprocessed_posts'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
df['preprocessed_posts'] = detokenized_doc
df['preprocessed_posts']



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        recently relationship boyfriend friend attache...
1        cheers evening opened elevator really beautifu...
2        titles depressed worthless around everyone par...
3        suffering energy exaggerating taking cooking c...
4        already stupid enough optimism foolish atleast...
                               ...                        
38028    someone please anymore disappear absolutely mi...
38029    really depressed talked school corona happened...
38030    people quarantine realize stupid saying making...
38031    messed things girlfriend couple months boyfrie...
38032    suicide attempt survivor depression intrusive ...
Name: preprocessed_posts, Length: 38033, dtype: object

**Document Term Matrix**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
dtMatrix = vectorizer.fit_transform(df['preprocessed_posts'])
dtMatrix.get_shape()

(13170, 19247)

**Applying LSA**

In [14]:
# Universally defining the final number of expected topics
numcomp=10
topic_num=[]

for tnum in range(1,numcomp+1):
  topic_num.append("topic_"+str(tnum))

# SVD represent documents and terms in vectors 
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=numcomp, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(dtMatrix)

# Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = topic_num)
topic_encoded_df["posts"] = df['preprocessed_posts']

# Features or words used as features 
dictionary = vectorizer.get_feature_names_out()

# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = topic_num, columns = (dictionary)).T

disp=["posts"]
disp.append(topic_num)
display(topic_encoded_df[topic_num])

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10
0,0.2344899613925449,-0.0397211058655747,0.0834744480538878,0.0155434220287317,0.0100122685222202,-0.0648388126702545,-0.0062955051430784,-0.0291539422464727,-0.0029388435874853,0.0155639638029029
1,0.0828791854205308,-0.0430068395500974,-0.0031255904060064,-0.0361769854477301,0.1172447207354869,-0.0136803209512675,-0.0114037789551716,-0.0635150989833826,0.1256949598254672,0.0783772698185123
2,0.1889357549829164,0.0650131641741568,0.0945566255700975,-0.0443828252339057,0.0195385867958046,0.2121135096018658,-0.0723083588361236,0.0130604084864091,-0.0712262143275772,-0.0012436349709031
3,0.1702667197499023,-0.0476516217976419,-0.0975623089468651,0.2229140700848597,-0.0268208992595177,0.0175002601401984,0.0821601074935108,0.0135289931147375,0.0167978918793310,0.0027143987063223
4,0.0198834418781901,0.0024998178461308,0.0022397384404481,0.0042650954315337,0.0046732876599514,0.0006760768007399,-0.0037789824478644,0.0009527221889359,-0.0020355637370111,0.0015418410307685
...,...,...,...,...,...,...,...,...,...,...
13165,0.1455316305342489,-0.0945990616676289,0.0414227392731764,-0.0602027517046763,0.1351729762427321,0.1126605964394995,0.0069177923736942,-0.0314096883428797,0.0893559678390360,0.0255281383714036
13166,0.1168747152259505,-0.0373116679211846,0.0982819229484511,0.0264230246236357,-0.1022637863302113,-0.0308785181498761,0.0683692694303661,0.0163437318401034,0.0184488042102472,0.0271094431429287
13167,0.1493690434468878,-0.0079914060133955,-0.0580448592524330,0.1011445181480861,0.0273359602425331,-0.0210334371907597,-0.0347141964387057,0.0225832434320818,0.0186268198881083,-0.1378431621019999
13168,0.0329062684993492,-0.0270046045480584,0.0073630479910881,0.0031677108784970,-0.0142097459363069,-0.0141879550786713,0.0044098962277752,0.0085361298863530,0.0135689989224536,-0.0136782832165548


**Getting topics**

In [15]:
terms = vectorizer.get_feature_names_out()
for i, comp in enumerate(svd_model.components_):
    terms_in_comp = zip(dictionary, comp)
    sorted_terms = sorted(terms_in_comp, key=lambda x:x[1], reverse=True)[:10]
    topic_words = [t[0] for t in sorted_terms]
    print("Topic {}: {}".format(i+1, ", ".join(topic_words)))

Topic 1: really, people, depression, friends, things, anymore, depressed, feeling, better, school
Topic 2: people, friends, anymore, school, lonely, person, friend, family, social, talking
Topic 3: people, depression, depressed, better, things, feeling, anxiety, normal, understand, medication
Topic 4: anymore, better, depression, people, things, living, trying, getting, matter, thoughts
Topic 5: really, feeling, better, depressed, things, anymore, people, crying, lately, matter
Topic 6: depressed, anymore, friends, feeling, school, friend, lonely, reason, crying, suicidal
Topic 7: really, depression, anymore, people, anxiety, diagnosed, understand, severe, struggling, advice
Topic 8: school, parents, people, depressed, really, things, college, getting, motivation, working
Topic 9: feeling, school, thoughts, crying, suicidal, parents, people, suicide, normal, scared
Topic 10: better, school, feeling, depression, really, lonely, friends, people, anymore, getting


**Calculating inter topic correlation**

In [16]:
# Calculate the correlation between LSA components
corr_matrix = topic_encoded_df.corr()
print(corr_matrix)

                     topic_1             topic_2             topic_3  \
topic_1   1.0000000000000000  0.1117397705745573 -0.0092063298496432   
topic_2   0.1117397705745573  1.0000000000000000  0.0002828674031291   
topic_3  -0.0092063298496432  0.0002828674031291  1.0000000000000000   
topic_4  -0.0957366290811595  0.0029415382779974 -0.0002423557119667   
topic_5   0.0376092667749781 -0.0011555566441786  0.0000952072442206   
topic_6   0.0288966304355515 -0.0008878581306560  0.0000731513477118   
topic_7   0.0968340076437409 -0.0029752555822125  0.0002451337078645   
topic_8  -0.0016983802050984  0.0000521832702055 -0.0000042994217339   
topic_9  -0.0901693376721362  0.0027704814845668 -0.0002282622047472   
topic_10  0.1502621129148783 -0.0046168510539171  0.0003803860832222   

                     topic_4             topic_5             topic_6  \
topic_1  -0.0957366290811595  0.0376092667749781  0.0288966304355515   
topic_2   0.0029415382779974 -0.0011555566441786 -0.00088785813